In [1]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessingStopwords
from sentence_transformers import SentenceTransformer
import pandas as pd
from nltk.corpus import stopwords as stop_words
import nltk
nltk.download('stopwords')

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


/Users/kuangli/opt/anaconda3/envs/py39env/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kuangli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('/Users/kuangli/Desktop/UCD/Fall_2024/STA_221/BA_Romdom.csv', encoding='latin-1')
df

,Query Id,Query Name,Date,Title,Snippet,Full Text,Url,Domain,Sentiment,Emotion,...,Twitter Followers,Twitter Following,Twitter Reply Count,Twitter Reply to,Twitter Retweet of,Twitter Retweets,Twitter Tweets,Twitter Verified,Reach (new),Reddit Score
0,2000584754,kuangli2 Monitor,2020-01-01 12:56:33.000,Leave it to me to be sick like dog on New Years,Leave it to me to be sick like dog on New Years,Leave it to me to be sick like dog on New Years,http://twitter.com/jenai_xo/statuses/121235693...,twitter.com,negative,Disgust,...,374,346,0,NaN,NaN,0,1878,False,0.0,0
1,2000584754,kuangli2 Monitor,2020-01-01 14:51:19.000,RT @JamaicaGleaner A vendor was rushed to hosp...,RT @JamaicaGleaner A vendor was rushed to hosp...,RT @JamaicaGleaner A vendor was rushed to hosp...,http://twitter.com/adorewildcherry/statuses/12...,twitter.com,neutral,Fear,...,483,520,0,NaN,http://twitter.com/JamaicaGleaner/statuses/121...,0,15139,False,0.0,0
2,2000584754,kuangli2 Monitor,2020-01-01 13:44:13.000,RT @fisayosoyombo In the second installation o...,RT @fisayosoyombo In the second installation o...,RT @fisayosoyombo In the second installation o...,http://twitter.com/Tosho_1/statuses/1212368936...,twitter.com,neutral,NaN,...,138,316,0,NaN,http://twitter.com/fisayosoyombo/statuses/1211...,0,4125,False,0.0,0
3,2000584754,kuangli2 Monitor,2020-01-01 12:03:03.000,Happy New Year! ______\n\nMay you have good he...,Happy New Year! ______ May you have good healt...,Happy New Year! ______ May you have good healt...,http://twitter.com/BluePostal/statuses/1212343...,twitter.com,positive,Joy,...,410,715,0,NaN,NaN,0,309,False,0.0,0
4,2000584754,kuangli2 Monitor,2020-01-01 13:41:18.000,RT @jojoansett 2020 is the year of health. Hea...,RT @jojoansett 2020 is the year of health. Hea...,RT @jojoansett 2020 is the year of health. Hea...,http://twitter.com/_letmebegreat_/statuses/121...,twitter.com,positive,Joy,...,349,265,0,NaN,http://twitter.com/jojoansett/statuses/1211793...,0,16031,False,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41268,2000584754,kuangli2 Monitor,2021-06-22 22:35:25.000,Bro I use to sick slip it up and calling out f...,Bro I use to sick slip it up and calling out f...,Bro I use to sick slip it up and calling out f...,http://twitter.com/DecemberSands/statuses/1407...,twitter.com,positive,Disgust,...,535,532,0,NaN,NaN,0,82909,False,0.0,0
41269,2000584754,kuangli2 Monitor,2021-06-21 15:49:48.000,RT @opmthebahamas #HappeningNow Prime Minister...,RT @opmthebahamas #HappeningNow Prime Minister...,RT @opmthebahamas #HappeningNow Prime Minister...,http://twitter.com/Kg242/statuses/140700282101...,twitter.com,neutral,NaN,...,234,550,0,NaN,http://twitter.com/opmthebahamas/statuses/1406...,0,28171,False,0.0,0
41270,2000584754,kuangli2 Monitor,2021-06-19 08:51:15.000,"Myanmar: Timely support, action by UNSC 'reall...","...paramount,"" she said. ""We have an alarming ...","...paramount,"" she said. ""We have an alarming ...",https://www.bahamasnews.net/news/269958967/mya...,bahamasnews.net,negative,Fear,...,0,0,0,NaN,NaN,0,0,False,NaN,0
41271,2000584754,kuangli2 Monitor,2021-06-21 17:59:55.000,RT @opmthebahamas #HappeningNow Prime Minister...,RT @opmthebahamas #HappeningNow Prime Minister...,RT @opmthebahamas #HappeningNow Prime Minister...,http://twitter.com/lthrwlls/statuses/140703556...,twitter.com,neutral,NaN,...,225,1511,0,NaN,http://twitter.com/opmthebahamas/statuses/1406...,0,5714,False,0.0,0


In [3]:
documents = [line.strip() for line in df['Full Text']]

stopwords = list(stop_words.words("english"))

sp = WhiteSpacePreprocessingStopwords(documents, stopwords_list=stopwords)
preprocessed_documents, unpreprocessed_corpus, vocab, retained_indices = sp.preprocess()

In [4]:
tp = TopicModelDataPreparation("all-mpnet-base-v2")

training_dataset = tp.fit(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)
ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, n_components=20, num_epochs=10)
ctm.fit(training_dataset) # run the model
ctm.get_topic_lists(5)

/Users/kuangli/opt/anaconda3/envs/py39env/lib/python3.9/site-packages/contextualized_topic_models/utils/data_preparation.py:64: UserWarning: the longest document in your collection has 2962 words, the model instead truncates to 128 tokens.
  warnings.warn(
Batches: 100%|██████████| 207/207 [03:40<00:00,  1.07s/it]
Epoch: [10/10]	 Seen Samples: [412160/412510]	Train Loss: 110.40380950596021	Time: 0:01:12.725568: : 10it [12:18, 73.87s/it]
100%|██████████| 645/645 [01:07<00:00,  9.54it/s] 


[['coronavirus', 'world', 'per', 'virus', 'last'],
 ['yoall', 'rt', 'donot', 'mask', 'wear'],
 ['ten', 'considering', 'www', 'reached', 'immigration'],
 ['cases', 'covid', 'new', 'bahamas', 'confirmed'],
 ['people', 'itos', 'donot', 'get', 'make'],
 ['https', 'co', 'quarantine', 'rt', 'yungry'],
 ['ministry', 'jasperward', 'reported', 'cases', 'new'],
 ['hospital', 'year', 'family', 'told', 'son'],
 ['bahamas', 'minister', 'pm', 'minnis', 'prime'],
 ['mask', 'wearing', 'nose', 'face', 'police'],
 ['reopen', 'testing', 'traveling', 'starting', 'announces'],
 ['people', 'us', 'get', 'need', 'one'],
 ['pandemic', 'middle', 'global', 'hit', 'due'],
 ['https', 'co', 'covid', 'morningblend', 'rt'],
 ['quarantine', 'stay', 'home', 'got', 'day'],
 ['like', 'time', 'one', 'back', 'eyes'],
 ['said', 'health', 'vaccine', 'government', 'also'],
 ['god', 'thank', 'life', 'strength', 'mental'],
 ['coronavirus', 'rt', 'trump', 'https', 'co'],
 ['sick', 'fuck', 'shit', 'tired', 'fucking']]